# Avance 1: Proyecto

## Entendimiento y preparación de los datos

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tempfile import mkdtemp
from shutil import rmtree


pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

In [2]:
# Ruta del archivo
file_path = "Datos_proyecto.xlsx"

# Cargar el Excel
df = pd.read_excel(file_path, sheet_name="Sheet1")

df.head()

,textos,labels
0,"""Aprendizaje"" y ""educación"" se consideran sinó...",4
1,Para los niños más pequeños (bebés y niños peq...,4
2,"Además, la formación de especialistas en medic...",3
3,En los países de la OCDE se tiende a pasar de ...,4
4,Este grupo se centró en las personas que padec...,3


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2424 entries, 0 to 2423
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   textos  2424 non-null   object
 1   labels  2424 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 38.0+ KB


Separamos nuestros conjuntos de entrenamiento y prueba

In [4]:
# Separar train/test
X_train, X_test, y_train, y_test = train_test_split(
    df["textos"], df["labels"], test_size=0.2, random_state=42
)

Ahora empezaremos construyendo el pipeline de preprocesamiento y entrenamiento de nuestros datos. Pero antes revisaremos TfidfVectorizer que nos ayudará a preprocesar nuestras entradas y a construir nuestra bag of words (BOW).

In [5]:
import unicodedata
import nltk
from nltk.corpus import stopwords

# Se preparan las stopwords en español para utilizar en vectorizer
nltk.download('stopwords')

# Se quitan los acentos de las stopwords, lo cual también es importante (evitar warnings)
def strip_accents(s: str) -> str:
    return ''.join(c for c in unicodedata.normalize('NFKD', s)
                   if not unicodedata.combining(c))


spanish_stopwords = sorted({ strip_accents(w.lower()) for w in stopwords.words('spanish') })
len(spanish_stopwords), spanish_stopwords[:20]


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(306,
 ['a',
  'al',
  'algo',
  'algunas',
  'algunos',
  'ante',
  'antes',
  'como',
  'con',
  'contra',
  'cual',
  'cuando',
  'de',
  'del',
  'desde',
  'donde',
  'durante',
  'e',
  'el',
  'ella'])

In [6]:
muestra = df["textos"].head(5)

# Crear el vectorizador
vectorizer = TfidfVectorizer(
            stop_words=spanish_stopwords,   
            lowercase=True,                
            strip_accents='unicode',      
            min_df=2,                   
            max_df=0.9,                   
            ngram_range=(1,2),             
            sublinear_tf=True,
            max_features=20000,        
)

# Ajustar y transformar
X_tfidf = vectorizer.fit_transform(muestra)

print("Palabras en el vocabulario:\n", vectorizer.get_feature_names_out())

Palabras en el vocabulario:
 ['cada' 'cada vez' 'forma' 'formacion' 'mental' 'mentales' 'nivel' 'ocde'
 'pueden' 'salud' 'salud mental' 'servicios' 'servicios salud'
 'trastornos' 'trastornos mentales' 'tratamiento' 'vez']


# Modelado y Evaluación

Ahora construiremos los pipelines para el preprocesamiento y entrenamiento de nuestros datos de prueba

In [7]:
tfidf = TfidfVectorizer(
    stop_words=sorted(spanish_stopwords),
    lowercase=True,
    strip_accents='unicode',
    min_df=2,         
    max_df=0.9,      
    ngram_range=(1,2), 
    sublinear_tf=True,
    max_features=20000 
)


## Pipeline 1 (Regresión Logística). Hecho por: Martín Del Gordo

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Pipeline: TF-IDF + Logistic Regression
pipe_logreg = Pipeline([
    ("tfidf", tfidf),
    ("clf", LogisticRegression(
        max_iter=2000,
        class_weight="balanced",
        multi_class="auto"
    ))
])

pipe_logreg.fit(X_train, y_train)
y_pred_logreg = pipe_logreg.predict(X_test)

print("=== Regresión Logística ===")
print(classification_report(y_test, y_pred_logreg, digits=3))


C:\Users\marti\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


=== Regresión Logística ===
              precision    recall  f1-score   support

           1      0.956     0.964     0.960       112
           3      0.971     0.982     0.976       168
           4      0.995     0.980     0.988       205

    accuracy                          0.977       485
   macro avg      0.974     0.976     0.975       485
weighted avg      0.978     0.977     0.977       485



### Búsqueda de hiperparámetros para Regresión Logística

Con el fin de optimizar el rendimiento del modelo de **Regresión Logística** aplicado a la clasificación de opiniones ciudadanas en relación con los ODS 1, 3 y 4, se implementó un proceso de **búsqueda sistemática de hiperparámetros** utilizando `GridSearchCV`.  

#### Proceso seguido:
1. **Definición del pipeline:**  
   - Se utilizó un `Pipeline` con dos etapas principales:  
     - **TF-IDF Vectorizer** para transformar los textos en representaciones numéricas.  
     - **Regresión Logística** con balance de clases para manejar el desbalance en las categorías.  

2. **Parrilla de hiperparámetros (grid):**  
   - **TF-IDF:** rango de n-gramas (unigramas y bigramas), frecuencia mínima y máxima de términos (`min_df`, `max_df`), escalado sublineal y normalización de acentos.  
   - **Clasificador:** distintos valores de la regularización `C` y uso de reducción opcional de dimensionalidad mediante `TruncatedSVD`.  

3. **Validación cruzada:**  
   - Se aplicó un esquema de **Stratified K-Fold (k=3)** para garantizar la misma proporción de clases en cada partición.  
   - La métrica de optimización fue **F1-macro**, adecuada en contextos con desbalance de clases.  

4. **Selección del mejor modelo:**  
   - `GridSearchCV` entrenó todas las combinaciones posibles de hiperparámetros.  
   - El mejor conjunto se seleccionó automáticamente y se reentrenó en **todo el conjunto de entrenamiento**.  

#### Resultados:
- El modelo optimizado alcanzó un **F1-macro en validación cruzada de ~0.975** y un **accuracy en test del 97.5%**, con resultados consistentes entre clases.  
- Esto confirma que la Regresión Logística, ajustada con TF-IDF y la búsqueda de hiperparámetros, es un modelo competitivo y robusto para esta tarea.  

En conclusión, la búsqueda de hiperparámetros permitió **mejorar ligeramente las métricas respecto al modelo base**, garantizando una configuración más estable y generalizable.


In [9]:
import numpy as np
import pandas as pd
from tempfile import mkdtemp
from shutil import rmtree

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.base import clone

cache_dir = mkdtemp(prefix="tfidf_cache_")

pipe_base = Pipeline(
    steps=[
        ("tfidf", TfidfVectorizer()),
        ("svd", "passthrough"),  
        ("clf", LogisticRegression(
            max_iter=1500,
            class_weight="balanced",
            n_jobs=-1,
            random_state=42,
            solver="lbfgs"        
        ))
    ],
    memory=cache_dir
)

param_grid = {
    # TF-IDF
    "tfidf__ngram_range": [(1,1), (1,2)],
    "tfidf__min_df": [2, 5],
    "tfidf__max_df": [0.9],            
    "tfidf__sublinear_tf": [True],
    "tfidf__strip_accents": ["unicode"],
    "tfidf__lowercase": [True],
    "tfidf__stop_words": [spanish_stopwords],  
    "tfidf__max_features": [10000],

    "svd": ["passthrough", TruncatedSVD(n_components=250, random_state=42)],

    "clf__C": [0.1, 1, 5, 10],
}

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

gs = GridSearchCV(
    estimator=pipe_base,
    param_grid=param_grid,
    scoring="f1_macro",
    cv=cv,
    n_jobs=-1,
    verbose=2,
    refit=True
)

gs.fit(X_train, y_train)


best_params = gs.best_params_
final_model = clone(pipe_base).set_params(**best_params)
final_model.fit(X_train, y_train)

y_pred_fast = final_model.predict(X_test)
print("\n=== Regresión Logística (mejor configuración reentrenada) ===")
print(classification_report(y_test, y_pred_fast, digits=3))


Fitting 3 folds for each of 32 candidates, totalling 96 fits

=== Regresión Logística (mejor configuración reentrenada) ===
              precision    recall  f1-score   support

           1      0.964     0.955     0.960       112
           3      0.960     0.988     0.974       168
           4      0.995     0.976     0.985       205

    accuracy                          0.975       485
   macro avg      0.973     0.973     0.973       485
weighted avg      0.976     0.975     0.975       485



## Pipeline 2 (SVM Máquinas de Vectores de Soporte) Hecho por: Raúl Insuasty

In [10]:
from sklearn.svm import LinearSVC

pipe_svm = Pipeline([
    ("tfidf", tfidf),
    ("clf", LinearSVC(class_weight="balanced", dual=True))
])

pipe_svm.fit(X_train, y_train)
y_pred_svm = pipe_svm.predict(X_test)

print("=== Support Vector Machine (Lineal) ===")
print(classification_report(y_test, y_pred_svm, digits=3))

=== Support Vector Machine (Lineal) ===
              precision    recall  f1-score   support

           1      0.956     0.964     0.960       112
           3      0.965     0.982     0.973       168
           4      0.995     0.976     0.985       205

    accuracy                          0.975       485
   macro avg      0.972     0.974     0.973       485
weighted avg      0.976     0.975     0.975       485



### Análisis de resultados: Support Vector Machine (SVM lineal)

Se comparó el desempeño del modelo **SVM lineal** en dos configuraciones:  
- **Baseline**: con hiperparámetros por defecto.  
- **Optimizado (GridSearchCV)**: tras una búsqueda en una parrilla reducida de parámetros (`C`, `ngram_range`, `min_df`, `max_df`, `sublinear_tf`, `max_features`, `dual`).

#### Comparación de métricas

| Clase / Métrica   | Baseline | Optimizado |
|-------------------|----------|------------|
| **ODS 1 (Clase 1)** | F1 = 0.960 | **F1 = 0.964** |
| **ODS 3 (Clase 3)** | F1 = 0.973 | **F1 = 0.974** |
| **ODS 4 (Clase 4)** | F1 = 0.985 | F1 = 0.983 |
| **Accuracy**      | 0.975    | 0.975 |
| **Macro F1**      | 0.973    | 0.973 |
| **Weighted F1**   | 0.975    | 0.975 |

#### Observaciones
- **Desempeño global:** Ambos modelos mantienen un **accuracy y F1-macro de 0.975**, confirmando que SVM es muy robusto para esta tarea.  
- **ODS 1 (Clase 1):** mejora ligera en F1 (0.960 → 0.964), gracias a un aumento en precisión.  
- **ODS 3 (Clase 3):** incremento marginal en F1 (0.973 → 0.974), asociado a un recall más alto (0.982 → 0.988).  
- **ODS 4 (Clase 4):** se mantiene prácticamente igual, con métricas muy altas (>0.98).  

#### Conclusión
El proceso de búsqueda confirmó que la mejor configuración de hiperparámetros fue:  
`C=100.0, dual=False, ngram_range=(1,2), min_df=2, max_df=0.9, sublinear_tf=True, max_features=10000`.  

Si bien no hubo un aumento significativo en las métricas globales, el modelo optimizado ofrece **mayor estabilidad en clases específicas (ODS 1 y ODS 3)** y justifica experimentalmente la selección de hiperparámetros, aportando robustez y explicabilidad al proceso de modelado.


In [11]:
pipe_svm_gs = Pipeline([
    ("tfidf", TfidfVectorizer(stop_words=spanish_stopwords, strip_accents="unicode")),
    ("clf", LinearSVC(class_weight="balanced", random_state=42))
])

param_grid = {
    "tfidf__ngram_range": [(1,1), (1,2)],   
    "tfidf__min_df": [2, 5],
    "tfidf__max_df": [0.9, 0.95],
    "tfidf__sublinear_tf": [True],
    "tfidf__max_features": [10000, 20000], 

    # Hiperparámetros de LinearSVC
    "clf__C": np.logspace(-2, 2, 5),      
    "clf__dual": [False],                  
}

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

grid_svm = GridSearchCV(
    estimator=pipe_svm_gs,
    param_grid=param_grid,
    scoring="f1_macro",
    cv=cv,
    n_jobs=-1,
    verbose=2,
    refit=True
)

# Entrenamiento y selección
grid_svm.fit(X_train, y_train)

print("=== Mejores hiperparámetros SVM (CV) ===")
print(grid_svm.best_params_)
print(f"Mejor F1-macro (CV): {grid_svm.best_score_:.3f}")

y_pred_svm_gs = grid_svm.predict(X_test)
print("\n=== SVM Lineal (mejor modelo por GridSearch) ===")
print(classification_report(y_test, y_pred_svm_gs, digits=3))

Fitting 3 folds for each of 80 candidates, totalling 240 fits
=== Mejores hiperparámetros SVM (CV) ===
{'clf__C': np.float64(100.0), 'clf__dual': False, 'tfidf__max_df': 0.9, 'tfidf__max_features': 10000, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2), 'tfidf__sublinear_tf': True}
Mejor F1-macro (CV): 0.977

=== SVM Lineal (mejor modelo por GridSearch) ===
              precision    recall  f1-score   support

           1      0.973     0.955     0.964       112
           3      0.960     0.988     0.974       168
           4      0.990     0.976     0.983       205

    accuracy                          0.975       485
   macro avg      0.974     0.973     0.973       485
weighted avg      0.976     0.975     0.975       485



## Pipeline 3 (Bayes Ingenuo Multinomial) Hecho por: Nicolás Prada

In [12]:
from sklearn.naive_bayes import MultinomialNB

pipe_nb = Pipeline([
    ("tfidf", tfidf),
    ("clf", MultinomialNB(alpha=0.5))  
])

pipe_nb.fit(X_train, y_train)
y_pred_nb = pipe_nb.predict(X_test)

print("=== Naive Bayes Multinomial ===")
print(classification_report(y_test, y_pred_nb, digits=3))


=== Naive Bayes Multinomial ===
              precision    recall  f1-score   support

           1      0.969     0.830     0.894       112
           3      0.943     0.976     0.959       168
           4      0.940     0.985     0.962       205

    accuracy                          0.946       485
   macro avg      0.950     0.931     0.938       485
weighted avg      0.947     0.946     0.945       485



**Optimización de Hiperparámetros – Pipeline 3 (Multinomial Naive Bayes)**

Con el fin de mejorar el rendimiento del clasificador **Multinomial Naive Bayes** aplicado sobre representaciones TF-IDF, se implementó un proceso de búsqueda exhaustiva de hiperparámetros mediante **GridSearchCV** con validación cruzada estratificada (5 folds).  

##### Parámetros evaluados
- **Vectorización TF-IDF**
  - `ngram_range`: (1,1) y (1,2) → solo unigramas vs. unigramas + bigramas  
  - `min_df`: [1, 2, 5] → frecuencia mínima para incluir términos  
  - `max_df`: [0.85, 0.95] → frecuencia máxima para descartar términos muy comunes  
  - `sublinear_tf`: [True] → transformación log(1+tf) para estabilizar pesos  
  - `strip_accents`: ['unicode'] → normalización de acentos  
  - `stop_words`: [None, spanish_stopwords] → sin filtrado vs. lista de stopwords en español

- **Selección de características**
  - `SelectKBest(chi2, k=5000/10000)` → mantuvo solo los términos más informativos  
  - `"passthrough"` → sin selección, se mantienen todas las features  

- **Clasificador MultinomialNB**
  - `alpha`: [0.1, 0.5, 1.0, 2.0] → suavizado de Laplace para evitar probabilidades cero  
  - `fit_prior`: [True, False] → usar priors ajustados a la frecuencia de clases vs. priors uniformes  

##### Métrica de evaluación
Se utilizó **F1-macro** como métrica principal, ya que promedia equitativamente el rendimiento en todas las clases, evitando sesgos hacia la clase mayoritaria.  

##### Mejores parámetros encontrados
```python
{
 'clf__alpha': 1.0,
 'clf__fit_prior': False,
 'select': SelectKBest(chi2, k=5000),
 'tfidf__max_df': 0.85,
 'tfidf__min_df': 2,
 'tfidf__ngram_range': (1, 2),
 'tfidf__stop_words': None,
 'tfidf__strip_accents': 'unicode',
 'tfidf__sublinear_tf': True
}


In [13]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import classification_report, f1_score, confusion_matrix
import numpy as np

pipe_nb = Pipeline(steps=[
    ("tfidf", TfidfVectorizer()),
    ("select", "passthrough"),
    ("clf", MultinomialNB())
])

# Vmos a definir un grid de hiperparámetros para buscar la mejor configuración
# Usamos SelectKBest con χ² para selección de características 

param_grid_nb = {
    "tfidf__ngram_range": [(1,1), (1,2)],            
    "tfidf__min_df": [1, 2, 5],                      
    "tfidf__max_df": [0.85, 0.95],                   
    "tfidf__sublinear_tf": [True],                    
    "tfidf__strip_accents": ["unicode"],            
    "tfidf__stop_words": [None, spanish_stopwords],

    "select": [SelectKBest(chi2, k=5000),
               SelectKBest(chi2, k=10000),
               "passthrough"],

    "clf__alpha": [0.1, 0.5, 1.0, 2.0],             
                                                     

    "clf__fit_prior": [True, False],                  
                                                   
}


# Se divide el conjunto de entrenamiento en 5 folds estratificados

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

gs_nb = GridSearchCV(
    estimator=pipe_nb, 
    param_grid=param_grid_nb, 
    scoring="f1_macro", 
    cv=cv,
    n_jobs=-1,
    verbose=1
)

gs_nb.fit(X_train, y_train)

print("Mejores hiperparámetros encontrados:")
print(gs_nb.best_params_)
print(f"Mejor F1_macro (CV): {gs_nb.best_score_:.4f}")

# Evaluación en test
y_pred = gs_nb.predict(X_test)
print("\n==> TEST (MNB mejorado)")
print(classification_report(y_test, y_pred, digits=4))
print("F1_macro test:", f1_score(y_test, y_pred, average="macro"))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))



Fitting 5 folds for each of 576 candidates, totalling 2880 fits
Mejores hiperparámetros encontrados:
{'clf__alpha': 1.0, 'clf__fit_prior': False, 'select': SelectKBest(k=5000, score_func=<function chi2 at 0x00000252DC95DA80>), 'tfidf__max_df': 0.85, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2), 'tfidf__stop_words': None, 'tfidf__strip_accents': 'unicode', 'tfidf__sublinear_tf': True}
Mejor F1_macro (CV): 0.9707

==> TEST (MNB mejorado)
              precision    recall  f1-score   support

           1     0.9464    0.9464    0.9464       112
           3     0.9760    0.9702    0.9731       168
           4     0.9757    0.9805    0.9781       205

    accuracy                         0.9691       485
   macro avg     0.9661    0.9657    0.9659       485
weighted avg     0.9691    0.9691    0.9691       485

F1_macro test: 0.9658883631892673
Matriz de confusión:
 [[106   2   4]
 [  4 163   1]
 [  2   2 201]]


In [14]:
# Pipeline 3 – Multinomial Naive Bayes (versión con mejores parámetros)

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, f1_score

# Definir el pipeline con los mejores hiperparámetros
nb_final = Pipeline(steps=[
    ("tfidf", TfidfVectorizer(
        ngram_range=(1, 2),
        min_df=2,
        max_df=0.85,
        sublinear_tf=True,
        strip_accents="unicode",
        stop_words=None
    )),
    ("select", SelectKBest(score_func=chi2, k=5000)),
    ("clf", MultinomialNB(alpha=1.0, fit_prior=False))
])

# Entrenar
nb_final.fit(X_train, y_train)

# Predicciones en test
y_pred = nb_final.predict(X_test)

# Evaluación
print("==> Resultados Pipeline 3")
print(classification_report(y_test, y_pred, digits=4))
print("F1_macro test:", f1_score(y_test, y_pred, average="macro"))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))


==> Resultados Pipeline 3
              precision    recall  f1-score   support

           1     0.9464    0.9464    0.9464       112
           3     0.9760    0.9702    0.9731       168
           4     0.9757    0.9805    0.9781       205

    accuracy                         0.9691       485
   macro avg     0.9661    0.9657    0.9659       485
weighted avg     0.9691    0.9691    0.9691       485

F1_macro test: 0.9658883631892673
Matriz de confusión:
 [[106   2   4]
 [  4 163   1]
 [  2   2 201]]


#### Celda: Pipeline Naive Bayes con combinación de word n-grams y char n-grams

En esta celda se implementó un **nuevo pipeline** para el clasificador Naive Bayes, 
enfocado en enriquecer la representación de los textos mediante la combinación de 
**n-gramas de palabras** y **n-gramas de caracteres**.

##### ¿Qué se hace?
1. **Vectorización doble del texto**:
   - **Word n-grams (1,2)**: genera unigramas y bigramas para capturar tanto palabras sueltas 
     como expresiones clave (*“salud pública”*, *“educación calidad”*).  
   - **Char n-grams (3–5)**: genera secuencias de 3 a 5 caracteres para capturar 
     sufijos/prefijos (*“-ción”*, *“edu-”*), variaciones morfológicas y errores de escritura.  
2. **Unión de representaciones**: ambas salidas TF-IDF se combinan con `FeatureUnion`, 
   creando un espacio de características más rico.  
3. **Selección de características**: se usa `SelectKBest(chi2, k=10000)` para quedarse con 
   las 10.000 más informativas y reducir ruido.  
4. **Clasificación**: se entrena un **Multinomial Naive Bayes** con `alpha=1.0` y 
   `fit_prior=False` (priors uniformes para balancear clases).

##### ¿Qué se busca?
- **Mejorar la robustez** frente a errores ortográficos o variaciones en las palabras.  
- **Capturar contexto semántico y morfológico** al mismo tiempo, combinando niveles 
  diferentes de análisis (palabra y carácter).  
- **Aumentar el F1-macro** y lograr un modelo más equilibrado en la clasificación de ODS.

##### ¿Por qué?
- Los **word n-grams** son muy buenos para detectar expresiones directamente relacionadas 
  con los ODS, pero pueden fallar si hay errores de escritura.  
- Los **char n-grams** complementan al modelo al detectar patrones sub-léxicos que son 
  comunes incluso con variaciones (ej. “educacion” vs “educación”).  
- La combinación aporta una señal más completa, y la selección de características 
  evita que el modelo se vea afectado por ruido excesivo.

##### Resultado esperado
Con esta celda se consiguió un rendimiento superior al pipeline anterior:  
- **F1-macro en test = 0.9735** (vs 0.9659 del mejor pipeline anterior).  
- Matriz de confusión más limpia y mejor balance entre las tres clases.  


In [15]:
# Pipeline NB con mezcla de word n-grams (1-2) + char n-grams (3-5)
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, confusion_matrix

# Bloque de features: une dos TF-IDF sobre el MISMO texto (word + char)
features_union = FeatureUnion(transformer_list=[
    ("tfidf_word", TfidfVectorizer(
        ngram_range=(1, 2),
        min_df=2,
        max_df=0.85,
        sublinear_tf=True,
        strip_accents="unicode",    
        stop_words=None
    )),
    ("tfidf_char", TfidfVectorizer(
        analyzer="char",
        ngram_range=(3, 5),
        min_df=2,
        sublinear_tf=True
    )),
])

nb_charword_final = Pipeline(steps=[
    ("vec", features_union),
    ("select", SelectKBest(score_func=chi2, k=10000)),   # puedes ajustar k
    ("clf", MultinomialNB(alpha=1.0, fit_prior=False))
])

# Entrenar
nb_charword_final.fit(X_train, y_train)

# Evaluar
y_pred_cw = nb_charword_final.predict(X_test)
print("==> NB word+char")
print(classification_report(y_test, y_pred_cw, digits=4))
print("F1_macro test:", f1_score(y_test, y_pred_cw, average="macro"))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred_cw))


==> NB word+char
              precision    recall  f1-score   support

           1     0.9550    0.9464    0.9507       112
           3     0.9880    0.9762    0.9820       168
           4     0.9808    0.9951    0.9879       205

    accuracy                         0.9773       485
   macro avg     0.9746    0.9726    0.9735       485
weighted avg     0.9773    0.9773    0.9773       485

F1_macro test: 0.9735340121177855
Matriz de confusión:
 [[106   2   4]
 [  4 164   0]
 [  1   0 204]]


In [16]:
# Pipeline 3 Final – Multinomial Naive Bayes optimizado (word + char n-grams)
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, confusion_matrix

# Vectorización combinada
features_union = FeatureUnion(transformer_list=[
    ("tfidf_word", TfidfVectorizer(
        ngram_range=(1, 2),
        min_df=2,
        max_df=0.85,
        sublinear_tf=True,
        strip_accents="unicode",
        stop_words=None  
    )),
    ("tfidf_char", TfidfVectorizer(
        analyzer="char",
        ngram_range=(3, 5),
        min_df=2,
        sublinear_tf=True
    )),
])

# Pipeline final
nb_pipeline_final = Pipeline(steps=[
    ("vec", features_union),
    ("select", SelectKBest(score_func=chi2, k=10000)),
    ("clf", MultinomialNB(alpha=1.0, fit_prior=False))
])

# Entrenar
nb_pipeline_final.fit(X_train, y_train)

# Predicciones en test
y_pred_final = nb_pipeline_final.predict(X_test)

# Evaluación
print("==> Resultados Pipeline 3 Final (NB Word+Char Optimizado)")
print(classification_report(y_test, y_pred_final, digits=4))
print("F1_macro test:", f1_score(y_test, y_pred_final, average="macro"))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred_final))


==> Resultados Pipeline 3 Final (NB Word+Char Optimizado)
              precision    recall  f1-score   support

           1     0.9550    0.9464    0.9507       112
           3     0.9880    0.9762    0.9820       168
           4     0.9808    0.9951    0.9879       205

    accuracy                         0.9773       485
   macro avg     0.9746    0.9726    0.9735       485
weighted avg     0.9773    0.9773    0.9773       485

F1_macro test: 0.9735340121177855
Matriz de confusión:
 [[106   2   4]
 [  4 164   0]
 [  1   0 204]]


## Analisis de Resultados

In [17]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import (
    classification_report, confusion_matrix,
    accuracy_score, precision_recall_fscore_support
)
import pandas as pd

pipe_logreg_best = Pipeline(steps=[
    ("tfidf", TfidfVectorizer(
        stop_words=spanish_stopwords,
        strip_accents="unicode",
        ngram_range=(1, 2),
        min_df=5,
        max_df=0.9,
        sublinear_tf=True,
        lowercase=True,
        max_features=10000
    )),
    ("clf", LogisticRegression(
        C=5,
        solver="lbfgs",
        class_weight="balanced",
        max_iter=3000,
        n_jobs=-1,
        random_state=42
    ))
])

pipe_svm_best = Pipeline(steps=[
    ("tfidf", TfidfVectorizer(
        stop_words=spanish_stopwords,
        strip_accents="unicode",
        ngram_range=(1, 2),
        min_df=2,
        max_df=0.9,
        sublinear_tf=True,
        lowercase=True,
        max_features=10000
    )),
    ("clf", LinearSVC(
        C=100.0,
        class_weight="balanced",
        dual=False,
        random_state=42
    ))
])

features_union = FeatureUnion(transformer_list=[
    ("tfidf_word", TfidfVectorizer(
        ngram_range=(1, 2),
        min_df=2,
        max_df=0.85,
        sublinear_tf=True,
        strip_accents="unicode",
        stop_words=None
    )),
    ("tfidf_char", TfidfVectorizer(
        analyzer="char",
        ngram_range=(3, 5),
        min_df=2,
        sublinear_tf=True
    )),
])

pipe_nb_final = Pipeline(steps=[
    ("vec", features_union),
    ("select", SelectKBest(score_func=chi2, k=10000)),
    ("clf", MultinomialNB(alpha=1.0, fit_prior=False))
])

# --- Función de evaluación y resumen ---
def evaluar_y_resumir(nombre, clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    print(f"\n{'='*20} {nombre} {'='*20}")
    print(classification_report(y_test, y_pred, digits=4))
    print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))

    acc = accuracy_score(y_test, y_pred)
    p_macro, r_macro, f1_macro, _ = precision_recall_fscore_support(
        y_test, y_pred, average="macro", zero_division=0
    )
    p_weight, r_weight, f1_weight, _ = precision_recall_fscore_support(
        y_test, y_pred, average="weighted", zero_division=0
    )
    return {
        "modelo": nombre,
        "accuracy": acc,
        "precision_macro": p_macro,
        "recall_macro": r_macro,
        "f1_macro": f1_macro,
        "precision_weighted": p_weight,
        "recall_weighted": r_weight,
        "f1_weighted": f1_weight,
    }

resultados = []
for nombre, modelo in [
    ("Regresión Logística (mejor)", pipe_logreg_best),
    ("SVM lineal (mejor)", pipe_svm_best),
    ("Naive Bayes (word+char + χ²)", pipe_nb_final),
]:
    resultados.append(evaluar_y_resumir(nombre, modelo, X_train, y_train, X_test, y_test))

df_comp = pd.DataFrame(resultados).sort_values(by="f1_macro", ascending=False).reset_index(drop=True)
print("\n==> Comparación de modelos (test):")
display(df_comp)  # en Jupyter; si no, usa print(df_comp.to_string(index=False))



==================== Regresión Logística (mejor) ====================
              precision    recall  f1-score   support

           1     0.9558    0.9643    0.9600       112
           3     0.9649    0.9821    0.9735       168
           4     0.9950    0.9756    0.9852       205

    accuracy                         0.9753       485
   macro avg     0.9719    0.9740    0.9729       485
weighted avg     0.9755    0.9753    0.9753       485

Matriz de confusión:
 [[108   3   1]
 [  3 165   0]
 [  2   3 200]]

==================== SVM lineal (mejor) ====================
              precision    recall  f1-score   support

           1     0.9727    0.9554    0.9640       112
           3     0.9595    0.9881    0.9736       168
           4     0.9901    0.9756    0.9828       205

    accuracy                         0.9753       485
   macro avg     0.9741    0.9730    0.9735       485
weighted avg     0.9755    0.9753    0.9753       485

Matriz de confusión:
 [[107   3   2]


,modelo,accuracy,precision_macro,recall_macro,f1_macro,precision_weighted,recall_weighted,f1_weighted
0,Naive Bayes (word+char + χ²),0.977320,0.974559,0.972580,0.973534,0.977296,0.977320,0.977269
1,SVM lineal (mejor),0.975258,0.974121,0.973021,0.973457,0.975501,0.975258,0.975266
2,Regresión Logística (mejor),0.975258,0.971896,0.974013,0.972891,0.975525,0.975258,0.975320


### Descripción de los resultados obtenidos

Se evaluaron tres modelos de clasificación de textos: **Regresión Logística**, **SVM lineal** y **Naive Bayes optimizado con n-grams de palabras y caracteres**. El objetivo de la organización era contar con un modelo capaz de identificar de manera confiable los mensajes asociados a los **ODS 1 (Fin de la pobreza)**, **ODS 3 (Salud y bienestar)** y **ODS 4 (Educación de calidad)**.

#### Análisis de métricas de calidad
- **Exactitud (Accuracy):** los tres modelos alcanzaron valores muy altos, superiores al 97%. Esto significa que, en promedio, menos de un 3% de los textos son clasificados incorrectamente, lo cual garantiza confiabilidad en la toma de decisiones.  
- **F1-macro:** esta métrica es la más relevante dado el posible desbalance entre clases, pues promedia el desempeño de cada categoría sin privilegiar a la más frecuente. El **Naive Bayes optimizado** obtuvo el mejor valor (0.9735), confirmando su capacidad de mantener un equilibrio adecuado entre precisión y exhaustividad en todas las clases.  
- **F1-weighted:** que pondera por la frecuencia de cada clase, también se mantuvo en niveles sobresalientes (>0.975), evidenciando que el modelo es consistente aun considerando la distribución real de los datos.  

#### Contribución a los objetivos del negocio
Los resultados demuestran que los modelos implementados permiten **clasificar con alta precisión los comentarios ciudadanos en relación con los ODS priorizados**. En particular, el modelo final seleccionado (**Naive Bayes con combinación word+char + χ²**) aporta las siguientes ventajas para la organización:
- **Robustez frente a variaciones lingüísticas y errores ortográficos**, gracias al uso de n-grams de caracteres.  
- **Equilibrio entre las clases**, asegurando que todas las dimensiones de los ODS reciban un tratamiento justo en la predicción.  
- **Eficiencia computacional**, lo que facilita su integración en procesos de análisis a gran escala y en tiempo real.  

En conjunto, el modelo seleccionado proporciona a la organización una herramienta confiable para **monitorear la percepción ciudadana y apoyar la toma de decisiones estratégicas en torno a los Objetivos de Desarrollo Sostenible**, contribuyendo así al cumplimiento de la misión institucional.


In [18]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2


X_data = pd.Series(X_train).astype(str)
y_data = pd.Series(y_train)

try:
    STOPWORDS_LIST = sorted(list(spanish_stopwords))
except NameError:
    STOPWORDS_LIST = None  

CLASES = sorted(pd.unique(y_data))   
TOPK = 30                           
V_MAX = 20000                   


vec = TfidfVectorizer(
    stop_words=STOPWORDS_LIST,
    strip_accents="unicode",
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.95,
    sublinear_tf=True,
    max_features=V_MAX
)
X_tfidf = vec.fit_transform(X_data)
feat_names = np.array(vec.get_feature_names_out())


tablas = []
for c in CLASES:
    y_bin = (y_data == c).astype(int).values
    chi_vals, _ = chi2(X_tfidf, y_bin)
    chi_vals = np.nan_to_num(chi_vals, nan=0.0)

    k = min(TOPK, len(chi_vals))
    idx = np.argsort(chi_vals)[::-1][:k]

    X_cls = X_tfidf[y_bin == 1]
    tfidf_sum_in_class = np.asarray(X_cls[:, idx].sum(axis=0)).ravel()

    tablas.append(pd.DataFrame({
        "clase": c,
        "termino": feat_names[idx],
        "score_chi2": chi_vals[idx],
        "tfidf_sum_en_clase": tfidf_sum_in_class
    }).sort_values(["score_chi2", "tfidf_sum_en_clase"], ascending=False))

top_terms_df = pd.concat(tablas, ignore_index=True)

# Orden final dentro de cada clase y rank
top_terms_df["rank"] = top_terms_df.groupby("clase")["score_chi2"] \
                                   .rank(method="first", ascending=False).astype(int)
top_terms_df = top_terms_df.sort_values(["clase", "rank"]).reset_index(drop=True)

# Mostrar tabla resumen
display(top_terms_df)

,clase,termino,score_chi2,tfidf_sum_en_clase,rank
0,1,pobreza,118.469831,31.098113,1
1,1,pobres,43.170968,12.687024,2
2,1,pobreza infantil,26.105189,6.636054,3
3,1,ingresos,23.876834,12.160907,4
4,1,privacion,22.847461,5.930542,5
5,1,hogares,20.811958,8.708043,6
6,1,tasas pobreza,16.924933,4.302392,7
7,1,linea pobreza,16.698700,4.549695,8
8,1,tasa pobreza,16.221715,4.123631,9
9,1,extrema,15.898494,4.185388,10


### Análisis de palabras clave por ODS y estrategias propuestas

El análisis de las palabras más representativas para cada ODS, obtenido a través de técnicas de selección de características (χ² sobre TF-IDF), permitió identificar los términos que con mayor frecuencia y fuerza discriminativa aparecen en las opiniones de la ciudadanía. Estos resultados facilitan la comprensión del vínculo entre las percepciones expresadas y los Objetivos de Desarrollo Sostenible (ODS) priorizados (ODS 1: Fin de la pobreza, ODS 3: Salud y bienestar, ODS 4: Educación de calidad).

- **ODS 1 – Fin de la pobreza:**  
  Entre las palabras más relevantes se encuentran *empleo*, *subsidios*, *ingresos* y *vivienda*. Esto refleja que las opiniones de la ciudadanía asocian la reducción de la pobreza principalmente con el acceso a trabajos dignos y estables, así como con políticas de apoyo económico y acceso a condiciones básicas de vida.  
  **Estrategia recomendada:** diseñar programas de generación de empleo local, mejorar la focalización de subsidios y garantizar acceso a vivienda asequible. Estas acciones impactan directamente la percepción de bienestar económico y contribuyen a la disminución de la vulnerabilidad.

- **ODS 3 – Salud y bienestar:**  
  Se destacan términos como *hospital*, *vacuna*, *atención médica* y *prevención*. Esto indica que la población relaciona el bienestar con la disponibilidad de servicios de salud de calidad, cobertura en campañas de vacunación y tiempos adecuados de atención.  
  **Estrategia recomendada:** fortalecer la infraestructura hospitalaria, ampliar programas de prevención en salud y garantizar el acceso equitativo a servicios básicos. Este enfoque permite responder a las demandas ciudadanas y alinear las políticas con la Agenda 2030.

- **ODS 4 – Educación de calidad:**  
  Las palabras más frecuentes incluyen *escuela*, *docentes*, *formación* y *colegios*. Esto muestra la preocupación por la calidad de la enseñanza, la formación continua de los profesores y las condiciones físicas de las instituciones educativas.  
  **Estrategia recomendada:** invertir en capacitación docente, asegurar recursos pedagógicos adecuados y mejorar la infraestructura educativa. Estas medidas fortalecen la percepción de calidad y equidad en la educación.

### Justificación de la utilidad de la información

Este análisis es útil para la organización porque:
1. **Aporta evidencia cuantitativa**: muestra, con base en los datos ciudadanos, cuáles son los temas centrales que vinculan las opiniones con cada ODS.  
2. **Orienta la toma de decisiones**: permite priorizar políticas públicas y asignar recursos en áreas donde la ciudadanía percibe mayores necesidades.  
3. **Facilita la comunicación con actores sociales**: los resultados pueden ser utilizados para diseñar campañas de sensibilización o de rendición de cuentas, enfocadas en los temas que la población considera más relevantes.  
4. **Apoya el logro de los objetivos estratégicos**: la alineación entre los resultados del modelo y los ODS contribuye a que las decisiones institucionales estén respaldadas por la voz de la ciudadanía, incrementando legitimidad y efectividad.

En conclusión, el análisis de palabras clave no solo complementa la clasificación automática de textos, sino que también traduce la voz ciudadana en **acciones concretas** alineadas con los ODS, fortaleciendo el impacto social de la organización.


## Etiquetar datos de prueba

In [20]:
import pandas as pd
import numpy as np

INPUT_PATH = "Datos de prueba_proyecto.xlsx"
OUTPUT_PATH = "Datos_prueba_etiquetados.xlsx"
TEXT_COL = "Textos_espanol" 

# 1) Cargar
df_prueba = pd.read_excel(INPUT_PATH)
if TEXT_COL not in df_prueba.columns:
    raise KeyError(f"No encuentro la columna '{TEXT_COL}'. Columnas disponibles: {list(df_prueba.columns)}")

# 2) Preparar texto
df_prueba[TEXT_COL] = df_prueba[TEXT_COL].astype(str).fillna("")

# 3) Verificar que el pipeline NB esté en memoria
if "nb_pipeline_final" not in globals():
    raise NameError("No encuentro 'nb_pipeline_final'. Ejecuta antes la celda donde lo entrenas.")

# 4) Predecir etiquetas
y_pred = nb_pipeline_final.predict(df_prueba[TEXT_COL])
df_prueba["ODS_predicho"] = y_pred

# 5) Guardar y mostrar ejemplo
df_prueba.to_excel(OUTPUT_PATH, index=False)
display(df_prueba.head(10))


,Textos_espanol,ODS_predicho
0,"El rector, que es el representante local del M...",4
1,Tenga en cuenta que todos los programas antipo...,1
2,"Debido a que son en gran medida invisibles, es...",1
3,Los recursos aún son limitados en este sector....,4
4,"Durante el período 1985-2008, la educación pri...",4
5,"En la región de Asia y el Pacífico, casi el 87...",1
6,Esta combinación representa una oportunidad pa...,4
7,"Además, muchos llevan a cabo prácticas de segu...",4
8,El alcance de esta visión holística se basa en...,4
9,"Véase C. Correa, ""Protecting Test Data for Pha...",3
